In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re

In [2]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rahei\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\rahei\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\rahei\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rahei\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
df_query = pd.read_parquet('D:/FAST_NUCES/FYP/Code/SemanticSearch/Preimplemented/AmazonDataset/esci-data/shopping_queries_dataset/shopping_queries_dataset_examples.parquet')
df_items = pd.read_parquet('D:/FAST_NUCES/FYP/Code/SemanticSearch/Preimplemented/AmazonDataset/esci-data/shopping_queries_dataset/shopping_queries_dataset_products.parquet')
df_sources = pd.read_csv("D:/FAST_NUCES/FYP/Code/SemanticSearch/Preimplemented/AmazonDataset/esci-data/shopping_queries_dataset/shopping_queries_dataset_sources.csv")

In [4]:
# To drop rows with 'es' and 'jp' in the 'product_locale' column
condition = ~df_query['product_locale'].isin(['es', 'jp'])
filtered_df1 = df_query[condition]

In [5]:
condition = ~df_items['product_locale'].isin(['es', 'jp'])
filtered_df2 = df_items[condition]

In [6]:
# Merge df_examples with df_items on 'product_id'
df_combined = pd.merge(filtered_df1, filtered_df2, on='product_id', how='inner')

#Removing false category column
df_combined.drop(columns=['product_locale_y'], inplace=True)

#Renaming the category_y column to category
df_combined.rename(columns={'product_locale_x': 'product_locale'}, inplace=True) 

In [7]:
df_combined

,example_id,query,query_id,product_id,product_locale,esci_label,small_version,large_version,split,product_title,product_description,product_bullet_point,product_brand,product_color
0,0,revent 80 cfm,0,B000MOO21W,us,I,0,1,train,Panasonic FV-20VQ3 WhisperCeiling 190 CFM Ceil...,None,WhisperCeiling fans feature a totally enclosed...,Panasonic,White
1,291891,bathroom fan without light,13723,B000MOO21W,us,E,1,1,train,Panasonic FV-20VQ3 WhisperCeiling 190 CFM Ceil...,None,WhisperCeiling fans feature a totally enclosed...,Panasonic,White
2,1,revent 80 cfm,0,B07X3Y6B1V,us,E,0,1,train,Homewerks 7141-80 Bathroom Fan Integrated LED ...,None,OUTSTANDING PERFORMANCE: This Homewerk's bath ...,Homewerks,80 CFM
3,291854,bathroom fan with light,13722,B07X3Y6B1V,us,E,0,1,test,Homewerks 7141-80 Bathroom Fan Integrated LED ...,None,OUTSTANDING PERFORMANCE: This Homewerk's bath ...,Homewerks,80 CFM
4,2,revent 80 cfm,0,B07WDM7MQQ,us,E,0,1,train,Homewerks 7140-80 Bathroom Fan Ceiling Mount E...,None,OUTSTANDING PERFORMANCE: This Homewerk's bath ...,Homewerks,White
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1818820,2618551,골프공,130538,B082274MRL,us,E,0,1,train,2020 Callaway Chrome Soft X Golf Balls (Triple...,None,"The new Chrome Soft X is the modern Tour Ball,...",Callaway,Triple Track White
1818821,2618552,골프공,130538,B081B117DH,us,E,0,1,train,"Volvik Vimax Soft Green Golf Balls, Dozen",None,High energy soft core with soft feel and fast ...,Volvik,Green
1818822,2618558,골프공,130538,B00QN6LC9I,us,E,0,1,train,"Titleist Tour Speed Golf Balls, White, (One Do...",None,Exceptional Distance\nPenetrating Flight on Lo...,Titleist,White
1818823,2618560,골프공,130538,B07Z44KPHS,us,E,0,1,train,Cut Golf 2-Piece Ionomer Red Golf Balls,None,2-Piece Construction\nIonomer Cover\n60 Compre...,Cut Golf,White


In [8]:
df_combinedf2 = df_combined.drop_duplicates()

In [9]:
df_combined['combined_text'] = df_combined.apply(lambda row: row['product_title'] + ' ' + row['product_description'] if row['product_description'] is not None else row['product_title'], axis=1)

df_combined
#Take out exact matches only. 


,example_id,query,query_id,product_id,product_locale,esci_label,small_version,large_version,split,product_title,product_description,product_bullet_point,product_brand,product_color,combined_text
0,0,revent 80 cfm,0,B000MOO21W,us,I,0,1,train,Panasonic FV-20VQ3 WhisperCeiling 190 CFM Ceil...,None,WhisperCeiling fans feature a totally enclosed...,Panasonic,White,Panasonic FV-20VQ3 WhisperCeiling 190 CFM Ceil...
1,291891,bathroom fan without light,13723,B000MOO21W,us,E,1,1,train,Panasonic FV-20VQ3 WhisperCeiling 190 CFM Ceil...,None,WhisperCeiling fans feature a totally enclosed...,Panasonic,White,Panasonic FV-20VQ3 WhisperCeiling 190 CFM Ceil...
2,1,revent 80 cfm,0,B07X3Y6B1V,us,E,0,1,train,Homewerks 7141-80 Bathroom Fan Integrated LED ...,None,OUTSTANDING PERFORMANCE: This Homewerk's bath ...,Homewerks,80 CFM,Homewerks 7141-80 Bathroom Fan Integrated LED ...
3,291854,bathroom fan with light,13722,B07X3Y6B1V,us,E,0,1,test,Homewerks 7141-80 Bathroom Fan Integrated LED ...,None,OUTSTANDING PERFORMANCE: This Homewerk's bath ...,Homewerks,80 CFM,Homewerks 7141-80 Bathroom Fan Integrated LED ...
4,2,revent 80 cfm,0,B07WDM7MQQ,us,E,0,1,train,Homewerks 7140-80 Bathroom Fan Ceiling Mount E...,None,OUTSTANDING PERFORMANCE: This Homewerk's bath ...,Homewerks,White,Homewerks 7140-80 Bathroom Fan Ceiling Mount E...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1818820,2618551,골프공,130538,B082274MRL,us,E,0,1,train,2020 Callaway Chrome Soft X Golf Balls (Triple...,None,"The new Chrome Soft X is the modern Tour Ball,...",Callaway,Triple Track White,2020 Callaway Chrome Soft X Golf Balls (Triple...
1818821,2618552,골프공,130538,B081B117DH,us,E,0,1,train,"Volvik Vimax Soft Green Golf Balls, Dozen",None,High energy soft core with soft feel and fast ...,Volvik,Green,"Volvik Vimax Soft Green Golf Balls, Dozen"
1818822,2618558,골프공,130538,B00QN6LC9I,us,E,0,1,train,"Titleist Tour Speed Golf Balls, White, (One Do...",None,Exceptional Distance\nPenetrating Flight on Lo...,Titleist,White,"Titleist Tour Speed Golf Balls, White, (One Do..."
1818823,2618560,골프공,130538,B07Z44KPHS,us,E,0,1,train,Cut Golf 2-Piece Ionomer Red Golf Balls,None,2-Piece Construction\nIonomer Cover\n60 Compre...,Cut Golf,White,Cut Golf 2-Piece Ionomer Red Golf Balls


In [10]:
condition = ~df_combined['esci_label'].isin(['I', 'S','C'])
df_combined = df_combined[condition]

In [11]:
df_combined

,example_id,query,query_id,product_id,product_locale,esci_label,small_version,large_version,split,product_title,product_description,product_bullet_point,product_brand,product_color,combined_text
1,291891,bathroom fan without light,13723,B000MOO21W,us,E,1,1,train,Panasonic FV-20VQ3 WhisperCeiling 190 CFM Ceil...,None,WhisperCeiling fans feature a totally enclosed...,Panasonic,White,Panasonic FV-20VQ3 WhisperCeiling 190 CFM Ceil...
2,1,revent 80 cfm,0,B07X3Y6B1V,us,E,0,1,train,Homewerks 7141-80 Bathroom Fan Integrated LED ...,None,OUTSTANDING PERFORMANCE: This Homewerk's bath ...,Homewerks,80 CFM,Homewerks 7141-80 Bathroom Fan Integrated LED ...
3,291854,bathroom fan with light,13722,B07X3Y6B1V,us,E,0,1,test,Homewerks 7141-80 Bathroom Fan Integrated LED ...,None,OUTSTANDING PERFORMANCE: This Homewerk's bath ...,Homewerks,80 CFM,Homewerks 7141-80 Bathroom Fan Integrated LED ...
4,2,revent 80 cfm,0,B07WDM7MQQ,us,E,0,1,train,Homewerks 7140-80 Bathroom Fan Ceiling Mount E...,None,OUTSTANDING PERFORMANCE: This Homewerk's bath ...,Homewerks,White,Homewerks 7140-80 Bathroom Fan Ceiling Mount E...
5,3,revent 80 cfm,0,B07RH6Z8KW,us,E,0,1,train,Delta Electronics RAD80L BreezRadiance 80 CFM ...,This pre-owned or refurbished product has been...,Quiet operation at 1.5 sones\nBuilt-in thermos...,DELTA ELECTRONICS (AMERICAS) LTD.,White,Delta Electronics RAD80L BreezRadiance 80 CFM ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1818819,2618549,골프공,130538,B083T6LDTJ,us,E,0,1,train,"MIZUNO RB 566V GOLF BALL, White",None,3 piece construction\nMicro-dimple\nLow compre...,Mizuno,White,"MIZUNO RB 566V GOLF BALL, White"
1818820,2618551,골프공,130538,B082274MRL,us,E,0,1,train,2020 Callaway Chrome Soft X Golf Balls (Triple...,None,"The new Chrome Soft X is the modern Tour Ball,...",Callaway,Triple Track White,2020 Callaway Chrome Soft X Golf Balls (Triple...
1818821,2618552,골프공,130538,B081B117DH,us,E,0,1,train,"Volvik Vimax Soft Green Golf Balls, Dozen",None,High energy soft core with soft feel and fast ...,Volvik,Green,"Volvik Vimax Soft Green Golf Balls, Dozen"
1818822,2618558,골프공,130538,B00QN6LC9I,us,E,0,1,train,"Titleist Tour Speed Golf Balls, White, (One Do...",None,Exceptional Distance\nPenetrating Flight on Lo...,Titleist,White,"Titleist Tour Speed Golf Balls, White, (One Do..."


In [12]:
# Define text preprocessing function
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove special characters
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    text = ' '.join([word for word in text.split() if word not in stop_words])
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    text = ' '.join([lemmatizer.lemmatize(word) for word in text.split()])
    return text

In [ ]:
# Apply text preprocessing
df_combined['processed_text'] = df_combined['combined_text'].apply(preprocess_text)

In [ ]:
df_combined

In [ ]:
# Initialize the TF-IDF vectorizer
vectorizer = TfidfVectorizer()

In [ ]:
from sentence_transformers import SentenceTransformer, util
import torch
import pandas as pd

In [ ]:
# Initialize the SentenceTransformer model
embedder = SentenceTransformer('all-MiniLM-L6-v2')

# Encode all sentences in the dataset
corpus = df_combined['processed_text'].tolist()
corpus_embeddings = embedder.encode(corpus, convert_to_tensor=True)

# Query sentences:
queries = [
    '1 green decorative pillow not pillow case',
    '05 colorado seats not covers',
    '0ffice chairs without wheels'
]

In [ ]:
# Find the closest 10 sentences of the corpus for each query sentence based on cosine similarity
top_k = min(10, len(corpus))
for query in queries:
    query_embedding = embedder.encode(query, convert_to_tensor=True)

    # We use cosine-similarity and torch.topk to find the highest 5 scores
    cos_scores = util.cos_sim(query_embedding, corpus_embeddings)[0]
    top_results = torch.topk(cos_scores, k=top_k)

    print("\n\n======================\n\n")
    print("Query:", query)
    print("\nTop 5 most similar sentences in corpus:")

    for score, idx in zip(top_results[0], top_results[1]):
        print(corpus[idx], "(Score: {:.4f})".format(score))
